# 数据转换（transform）

对于 `Series` 与 `DataFrame` 对象，大多运算操作都是元素级别上的矢量化运算，结果长度并不改变（`n=>n`）；聚合运算操作（例如求均值、方差等）则是沿轴向对数据运算，返回一个标量值（`n=>1`）。在上一章，介绍在 Pandas 中使用 `map()`, `applymap()`, `apply()`等方法。其中：
- `s.map()`，`Series`对象的矢量化函数，在元素级应用字典或自定义函数等进行数据转换；
- `df.applymap()`，`DataFrame`对象的矢量化函数，在元素级上应用自定义函数；
- `apply()`，面向`Series`与`DataFrame`对象的通用矢量化方法，可以应用自定义函数。

本节介绍另外一个函数`transform()`，并使用该函数对数值型数据进行无量纲化。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 数值型数据无量纲化

数值型数据常用来表征不同特征属性，其计量单位和数量级都有很大差异，只有通过无量纲化才能实现数据之间的可比性。数据的无量纲化常用如下方法：
- 标准化。将原始数值型数据 $x$ 转变为均值为0标准差为1的变量：
$$ x' = \frac{x - \overline{x}}{\sigma} $$
其中 $\overline{x}$ 为均值，$\sigma$ 为标准差。
- 区间缩放法。把原始数值型数据范围在 `[0,1]` 之间的变量:
$$ x' = \frac{x - \min(x)}{\max(x) - \min(x)}$$

下面实现标准化与区间缩放法：

In [2]:
def zscore(x):
    """标准化"""
    return (x - x.mean())/ x.var()


def scale(x):
    """区间缩放法"""
    return (x - x.min())/ (x.max() - x.min())

例如，下面创建一个考试成绩的`DataFrame`对象：

In [3]:
data = {
    'English': np.random.randint(0, 5, 16),   # 5分制
    'Python': np.random.randint(0, 100, 16),  # 100分制
    'Math': np.random.randint(0, 150, 16),   # 150分制
}
df = pd.DataFrame(data)
df.head()

,English,Python,Math
0,2,43,56
1,4,74,42
2,3,57,73
3,1,87,108
4,1,14,27


`df.applymap()`方法是在元素级上应用自定义函数，无法应用标准化函数：

In [4]:
# 下面语句会抛出异常
# df.applymap(zscore)

可以使用`apply()`函数来进行数据标准化：

In [5]:
# 对数据进行标准化
df.apply(zscore)

,English,Python,Math
0,0.25,-0.007273,-0.005419
1,1.25,0.024938,-0.013005
2,0.75,0.007273,0.003793
3,-0.25,0.038446,0.022758
4,-0.25,-0.037406,-0.021133
5,0.25,-0.037406,-0.019507
6,-0.75,-0.005195,0.035221
7,-0.25,-0.043641,-0.009212
8,-0.75,0.039485,0.018965
9,-0.25,-0.040524,0.027635


## `transform()`使用

`Series`与`DataFrame`对象都提供有`transform()`方法，应用与数据转换。其使用语法为：
```
df.transform(func, *args, **kwargs)
```
主要参数
- `func`，可调用函数，字符串，字典或列表
- `args`，函数参数
- `kwargs`，函数关键字参数

`transform()`的功能与`apply()`类似，但其主要用于则返回与原对象维度一样的结果，其则有如下限制：
- 调用函数可以产生一个标量值，然后进行广播扩展
- 调用函数可以产生原对象维度相同的对象
- 不能改变输入数组

下面使用`transform()`来对数据进行无量纲化：

In [6]:
df.transform(scale)

,English,Python,Math
0,0.50,0.416667,0.389706
1,1.00,0.785714,0.286765
2,0.75,0.583333,0.514706
3,0.25,0.940476,0.772059
4,0.25,0.071429,0.176471
5,0.50,0.071429,0.198529
6,0.00,0.440476,0.941176
7,0.25,0.000000,0.338235
8,0.00,0.952381,0.720588
9,0.25,0.035714,0.838235


可以使用字典方式，为各列指定应用函数：

In [7]:
df.transform({'English': zscore, 'Python': scale})

,English,Python
0,0.25,0.416667
1,1.25,0.785714
2,0.75,0.583333
3,-0.25,0.940476
4,-0.25,0.071429
5,0.25,0.071429
6,-0.75,0.440476
7,-0.25,0.000000
8,-0.75,0.952381
9,-0.25,0.035714


`transform()`的运行结果与使用`apply()`的结果一样。不过，`apply()`可以应用聚合函数，而`transform()`则不可以，例如应用聚合函数（`mean`）：

In [8]:
# 使用聚合函数
df.apply('mean')

English     1.5
Python     50.0
Math       66.0
dtype: float64

In [9]:
# 使用聚合函数抛出异常
# df.transform(np.mean)

使用 `transform()` 是，如果指定列传入函数返回一个标量值，那么该值会在列上进行广播：

In [10]:
def datarange(x):
    return sum(x)

df.transform({'English': zscore, 'Python': scale, 'Math': datarange})

,English,Python,Math
0,0.25,0.416667,1056
1,1.25,0.785714,1056
2,0.75,0.583333,1056
3,-0.25,0.940476,1056
4,-0.25,0.071429,1056
5,0.25,0.071429,1056
6,-0.75,0.440476,1056
7,-0.25,0.000000,1056
8,-0.75,0.952381,1056
9,-0.25,0.035714,1056
